# 计算BOF和TDA所需要的时间

## 一些准备工作

### 一个计算运行时间的函数

In [19]:
import time

def coumpte_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function {func.__name__} took {execution_time} seconds to execute.")
        return result
    return wrapper


### 一个张量列表

In [20]:
import torch
# 创建一个空列表用于存放张量
tensor_list = []

# 定义张量形状
tensor_shape = (5000, 32*32*3)

# 生成5000个符合高斯分布的张量并添加到列表中
for _ in range(1):
    # 使用torch.randn生成符合标准正态分布的张量
    tensor = torch.randn(tensor_shape)
    tensor_list.append(tensor)

## 计算BOF消耗的时间

### 定义一个计算BOF的实例函数

In [21]:
from BOF.get_rank_from_matrix import Effective_Ranks
import torch



def get_BOF(tensor_list:list = []):
    # 现在tensor_list里面有5000个符合高斯分布的张量，每个张量形状为(5000, 32*32*3)
        

    results = []
    for image_matrix in tensor_list:
        # 检查张量形状
        if image_matrix.size(0) > image_matrix.size(1):
            image_matrix = image_matrix.T  # 如果 m > n，执行转置操作
        
        get_rank = Effective_Ranks(image_matrix)
        r0 = get_rank.r0
        R0 = get_rank.R0
        rk_max_index = get_rank.rk_max_index
        rk_max = get_rank.rk_max_value
        Rk_max = get_rank.Rk_value_max_rk_index

        results.append({"isic": {"r0": r0, "R0": R0, "rk_max_index": rk_max_index, "rk_max": rk_max, "Rk_max": Rk_max}})
    
    return results


In [22]:
# get_BOF()
# 使用function_A包装function_B
timed_function_B = coumpte_time(get_BOF)

# 调用timed_function_B，并传入参数
timed_function_B(tensor_list)  # 传入函数B所需的参数


torch.Size([3072, 5000]) torch.float32
(3072, 3072) float64
Function get_BOF took 3.4863224029541016 seconds to execute.


[{'isic': {'r0': 971.2970274901704,
   'R0': 1902.8135538809415,
   'rk_max_index': 189,
   'rk_max': 1033.3345264439295,
   'Rk_max': 1849.0088594414212}}]

## 计算TDA需要的时间

### 定义一个计算TDA的实例函数

In [23]:
from ripser import ripser

def get_TDA(tensor_list:list = []):

    # 现在tensor_list里面有5000个符合高斯分布的张量，每个张量形状为(5000, 32*32*3)
    
    results = []

    for image_matrix in tensor_list:
        TDA = ripser(X=image_matrix, maxdim=1)
        results.append(TDA['dgms'])

    return results
        

In [24]:
# get_BOF()
# 使用function_A包装function_B
timed_function_B = coumpte_time(get_TDA)

# 调用timed_function_B，并传入参数
timed_function_B(tensor_list)  # 传入函数B所需的参数


Function get_TDA took 37.43271732330322 seconds to execute.


[[array([[2.86102295e-06, 7.32283859e+01],
         [3.43852162e-06, 7.32453766e+01],
         [3.56832265e-06, 7.33399277e+01],
         ...,
         [2.52318500e-06, 7.70209656e+01],
         [4.26496126e-06, 7.70241852e+01],
         [0.00000000e+00,            inf]]),
  array([[77.14601898, 77.18371582],
         [77.04654694, 77.14965057],
         [77.04564667, 77.08319092],
         ...,
         [74.26155853, 75.28795624],
         [74.24530029, 75.20455933],
         [74.2314682 , 75.29425812]])]]